In [1]:
import numpy as np
import pydot
from IPython.display import HTML, SVG, display
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    GenerateHtml,
    InverseDynamicsController,
    PidController,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    Simulator,
    StartMeshcat,
    SceneGraph,
    PassThrough,
    Demultiplexer,
)

import catBot

ModuleNotFoundError: No module named 'catBot'

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [3]:
time_step = 1e-3

builder = DiagramBuilder()

# -- Add original plant -- #
# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
# Note that we parse into both the plant and the scene_graph here.
model = Parser(plant, scene_graph).AddModelFromFile(
    "../models/singleAxisCatBot.urdf")
plant.Finalize()

# -- Add controller plant -- #
controller_plant = MultibodyPlant(time_step=time_step)
model = Parser(controller_plant).AddModelFromFile(
    "../models/singleAxisCatBot.urdf")
controller_plant.Finalize()

# -- Add visualizer -- #
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

meshcat.ResetRenderMode()
meshcat.DeleteAddedControls()

# -- Add controller -- #
# Create a PID controller for each joint.
num_model_actuators = controller_plant.num_actuators()

state_projection_matrix = np.array([
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],

    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
])

kp = [1] * num_model_actuators 
ki = [0] * num_model_actuators 
kd = [20] * num_model_actuators 
catbot_controller = builder.AddSystem(
    PidController(state_projection_matrix, kp, ki, kd)
)

catbot_controller.set_name("catbot_controller")
builder.Connect(
    plant.get_state_output_port(model),
    catbot_controller.get_input_port_estimated_state(),
)
builder.Connect(
    catbot_controller.get_output_port_control(), 
    plant.get_actuation_input_port()
)

# -- Set up teleop wigids -- #

q0 = [0.0, 0.0, 0.0, 0.0, 0.0]

teleop = builder.AddSystem(
    MeshcatPoseSliders(
        meshcat,
        min_range=MeshcatPoseSliders.MinRange(roll=0,
                                                pitch=-0.5,
                                                yaw=-np.pi,
                                                x=-0.4,
                                                y=-0.4,
                                                z=0.0),
        max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi,
                                                pitch=np.pi,
                                                yaw=np.pi,
                                                x=0.4,
                                                y=0.25,
                                                z=0.75),
        body_index=plant.GetBodyByName("panda_link8").index(),
        # It seems that value is set by the default joint positions, not here.
        value=MeshcatPoseSliders.Value(roll=0.0,  # idk if this part works...  
                                    pitch=0.0,
                                    yaw=0.0,
                                    x=0.0,
                                    y=0.0,
                                    z=0.5)))